In [ ]:
#To download images of person alone , we can set number of images we want to download too.
import os
import requests
from pycocotools.coco import COCO
from tqdm import tqdm

# Define paths
annotations_file = '/path/to/person_keypoints_train2017.json'  # Enter Path to COCO annotations
output_dir = '/path/to/coco/coco_person_images'  # Enter path to directory to save images

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load COCO annotations
coco = COCO(annotations_file)

# Get all person category image IDs
cat_ids = coco.getCatIds(catNms=['person'])
img_ids = coco.getImgIds(catIds=cat_ids)

# Filter images to only include those that contain only person keypoints
filtered_img_ids = []
for img_id in tqdm(img_ids):
    ann_ids = coco.getAnnIds(imgIds=img_id, catIds=cat_ids)
    anns = coco.loadAnns(ann_ids)

    # Check if all annotations in this image are persons
    if all(ann['category_id'] == 1 for ann in anns):  # COCO 'person' category_id is 1
        filtered_img_ids.append(img_id)

    if len(filtered_img_ids) >= 15000: #Enter number of person images you want
        break

# Download images
for img_id in tqdm(filtered_img_ids):
    img_info = coco.loadImgs(img_id)[0]
    img_url = img_info['coco_url']
    img_filename = os.path.join(output_dir, img_info['file_name'])

    if not os.path.exists(img_filename):
        # Download and save the image
        img_data = requests.get(img_url).content
        with open(img_filename, 'wb') as f:
            f.write(img_data)

print(f'Downloaded {len(filtered_img_ids)} images to {output_dir}')
